# 🔎 Clarity AI | Image Upscaler & Enhancer

### Free and open-source Magnific Alternative for Google Colab

[![GitHub](https://img.shields.io/badge/GitHub-philz1337x/clarity--upscaler-blue?logo=github)](https://github.com/philz1337x/clarity-upscaler)
[![Website](https://img.shields.io/badge/Website-ClarityAI.co-blueviolet)](https://clarityai.co/)

---

**Instrucciones:**
1. Ejecuta las celdas en orden
2. La instalación puede tomar 5-10 minutos en la primera ejecución
3. Una vez completada la instalación, se abrirá la interfaz web
4. ¡Disfruta haciendo upscaling de tus imágenes con IA!

---


## 📋 Paso 1: Verificar GPU y Preparar Entorno

In [ ]:
# Verificar GPU disponible
import torch
import os

print("🔧 Verificando configuración del sistema...")
print(f"Python version: {torch.__version__}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU detectada: {gpu_name}")
    print(f"🚀 Memoria GPU: {gpu_memory:.1f} GB")
else:
    print("⚠️  ¡ADVERTENCIA! No se detectó GPU. Asegúrate de tener GPU habilitada en Colab.")
    print("Ve a Runtime → Change runtime type → Hardware accelerator → GPU")

# Configurar variables de entorno
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

print("\n🎯 Entorno configurado correctamente!")

## 🛠️ Paso 2: Instalación de Dependencias

In [ ]:
# Actualizar pip y instalar dependencias básicas
print("📦 Instalando dependencias básicas...")
!pip install -q --upgrade pip
!pip install -q wheel setuptools

# Instalar PyTorch con soporte CUDA (optimizado para Colab)
print("🔥 Instalando PyTorch con soporte CUDA...")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Instalar otras dependencias críticas
print("⚡ Instalando dependencias de ML y visión...")
!pip install -q transformers==4.30.2
!pip install -q accelerate
!pip install -q diffusers
!pip install -q xformers --index-url https://download.pytorch.org/whl/cu118

print("✅ Dependencias básicas instaladas!")

## 📥 Paso 3: Clonar Repositorio e Instalar Dependencias Específicas

In [ ]:
import os
import subprocess

# Cambiar al directorio temporal de Colab
os.chdir('/content')

# Clonar el repositorio si no existe
if not os.path.exists('clarity-upscaler'):
    print("📂 Clonando repositorio Clarity-Upscaler...")
    !git clone https://github.com/philz1337x/clarity-upscaler.git
else:
    print("📁 Repositorio ya existe, actualizando...")
    os.chdir('clarity-upscaler')
    !git pull
    os.chdir('/content')

# Cambiar al directorio del proyecto
os.chdir('/content/clarity-upscaler')

print("✅ Repositorio listo!")

In [ ]:
# Instalar dependencias específicas del proyecto
print("📋 Instalando requirements.txt...")

# Leer e instalar requirements con manejo de errores
try:
    with open('requirements.txt', 'r') as f:
        requirements = f.read().splitlines()
    
    # Filtrar líneas vacías y comentarios
    requirements = [req.strip() for req in requirements if req.strip() and not req.startswith('#')]
    
    # Instalar cada requirement individualmente para mejor control de errores
    failed_packages = []
    for req in requirements:
        try:
            print(f"Installing: {req}")
            result = subprocess.run(['pip', 'install', '-q', req], 
                                  capture_output=True, text=True, timeout=300)
            if result.returncode != 0:
                print(f"⚠️  Warning: Failed to install {req}")
                failed_packages.append(req)
        except Exception as e:
            print(f"⚠️  Error installing {req}: {str(e)}")
            failed_packages.append(req)
    
    if failed_packages:
        print(f"\n⚠️  Packages que fallaron: {failed_packages}")
        print("🔄 Intentando instalación alternativa...")
        for pkg in failed_packages:
            !pip install -q --no-deps {pkg}
    
except Exception as e:
    print(f"Error reading requirements.txt: {e}")
    # Fallback: instalar dependencias críticas manualmente
    critical_deps = [
        'gradio==3.41.2', 'fastapi', 'pillow', 'numpy', 'opencv-python',
        'scipy', 'scikit-image', 'omegaconf', 'einops', 'safetensors'
    ]
    for dep in critical_deps:
        !pip install -q {dep}

print("\n✅ Dependencias instaladas (con posibles warnings menores)!")

## 🤖 Paso 4: Descargar Modelos Necesarios

In [ ]:
import os
import urllib.request
from pathlib import Path

print("🤖 Descargando modelos necesarios...")

# Crear directorios necesarios
os.makedirs('models/Stable-diffusion', exist_ok=True)
os.makedirs('models/ControlNet', exist_ok=True)
os.makedirs('models/Lora', exist_ok=True)
os.makedirs('models/ESRGAN', exist_ok=True)

# Ejecutar el script de descarga de pesos si existe
if os.path.exists('download_weights.py'):
    print("⬇️  Ejecutando download_weights.py...")
    try:
        !python download_weights.py
    except Exception as e:
        print(f"⚠️  Warning en download_weights.py: {e}")

# Verificar si se descargaron modelos básicos
model_files = list(Path('models').rglob('*.safetensors')) + list(Path('models').rglob('*.ckpt'))
if model_files:
    print(f"✅ Se encontraron {len(model_files)} archivos de modelo")
else:
    print("⚠️  No se encontraron modelos específicos, pero el sistema puede descargarlos automáticamente")

print("🎯 Preparación de modelos completada!")

## ⚙️ Paso 5: Configuración para Google Colab

In [ ]:
import os
import sys

# Agregar el directorio actual al path de Python
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

# Configurar variables de entorno específicas para Colab
os.environ['GRADIO_SERVER_NAME'] = '0.0.0.0'
os.environ['GRADIO_SERVER_PORT'] = '7860'
os.environ['COLAB_TPU_ADDR'] = ''
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Configuraciones de memoria para optimizar en Colab
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Configurar para que no abra automáticamente el navegador
os.environ['SD_WEBUI_RESTARTING'] = '1'

print("⚙️  Configuración de Colab aplicada")
print(f"📍 Directorio de trabajo: {current_dir}")
print("✅ Listo para lanzar la interfaz!")

## 🚀 Paso 6: Lanzar Clarity Upscaler

In [ ]:
import sys
import os
import subprocess
import threading
import time

print("🚀 Lanzando Clarity Upscaler...")
print("⏰ Esto puede tomar un momento para cargar todos los modelos...")

# Cambiar al directorio correcto
os.chdir('/content/clarity-upscaler')

# Crear un script de lanzamiento personalizado para Colab
launch_script = '''
import os
import sys

# Configurar argumentos de línea de comandos para Colab
sys.argv = [
    "webui.py",
    "--listen",
    "--port", "7860",
    "--share",
    "--enable-insecure-extension-access",
    "--no-half-vae",
    "--no-download-sd-model",
    "--opt-split-attention",
    "--disable-console-progressbars"
]

# Importar y ejecutar el webui principal
exec(open('webui.py').read())
'''

# Escribir el script de lanzamiento
with open('launch_colab.py', 'w') as f:
    f.write(launch_script)

try:
    # Lanzar la aplicación
    print("🔄 Iniciando servidor...")
    process = subprocess.Popen(
        [sys.executable, 'launch_colab.py'],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )
    
    # Función para leer la salida en tiempo real
    def read_output():
        for line in iter(process.stdout.readline, ''):
            print(line.strip())
            if 'Running on public URL:' in line or 'public URL' in line:
                print("\n🎉 ¡Clarity Upscaler está listo!")
                print("🌐 Haz clic en el enlace público arriba para acceder a la interfaz")
                print("📝 Nota: La primera vez puede tardar más en cargar los modelos")
    
    # Ejecutar lectura en hilo separado
    thread = threading.Thread(target=read_output)
    thread.daemon = True
    thread.start()
    
    # Esperar un momento para que el proceso inicie
    time.sleep(10)
    
    print("\n⏳ Si no ves la URL pública después de unos minutos, revisa los logs arriba...")
    print("🔧 El proceso puede tardar en descargar modelos la primera vez")
    
    # Mantener el proceso corriendo
    try:
        process.wait()
    except KeyboardInterrupt:
        print("\n🛑 Deteniendo servidor...")
        process.terminate()
        
except Exception as e:
    print(f"❌ Error al lanzar la aplicación: {e}")
    print("\n🔧 Intentando método alternativo...")
    
    # Método alternativo: lanzar directamente webui.py con argumentos
    !python webui.py --listen --port 7860 --share --enable-insecure-extension-access --no-half-vae

## 🔧 Paso 7: Troubleshooting y Comandos Útiles

In [ ]:
# Ejecutar esta celda solo si tienes problemas con el lanzamiento anterior

import os
import torch

print("🔧 Información de diagnóstico:")
print(f"📍 Directorio actual: {os.getcwd()}")
print(f"🐍 Python path: {os.environ.get('PYTHONPATH', 'No configurado')}")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"⚡ CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Memoria GPU libre: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Verificar archivos críticos
critical_files = ['webui.py', 'requirements.txt', 'modules']
print("\n📁 Verificando archivos críticos:")
for file in critical_files:
    exists = os.path.exists(file)
    print(f"{'✅' if exists else '❌'} {file}: {'Existe' if exists else 'No encontrado'}")

print("\n💡 Si sigues teniendo problemas:")
print("1. Reinicia el runtime (Runtime → Restart runtime)")
print("2. Ejecuta todas las celdas nuevamente en orden")
print("3. Asegúrate de tener GPU habilitada en Colab")

## 🎯 Método Alternativo de Lanzamiento (Solo si el anterior falla)

In [ ]:
# Método alternativo más directo
import os
import sys

print("🔄 Método alternativo de lanzamiento...")

os.chdir('/content/clarity-upscaler')

# Configurar argumentos
os.environ['COMMANDLINE_ARGS'] = '--listen --port 7860 --share --enable-insecure-extension-access --no-half-vae --opt-split-attention'

# Lanzar directamente con argumentos de línea de comandos
!python webui.py --listen --port 7860 --share --enable-insecure-extension-access --no-half-vae --opt-split-attention --disable-console-progressbars

---

## 📖 Cómo Usar Clarity Upscaler

Una vez que la interfaz esté ejecutándose:

1. **Subir imagen**: Haz clic en el área de subida para seleccionar tu imagen
2. **Configurar parámetros**: Ajusta la configuración según tus necesidades
3. **Generar**: Haz clic en generar y espera el resultado
4. **Descargar**: Una vez completado, descarga tu imagen mejorada

### 💡 Consejos para Mejores Resultados:
- Usa imágenes de buena calidad como entrada
- Experimenta con diferentes configuraciones
- Ten paciencia: el proceso puede tomar varios minutos
- En Colab gratuito, las imágenes muy grandes pueden agotar la memoria

### 🚨 Limitaciones en Colab Gratuito:
- Memoria GPU limitada (~15GB)
- Tiempo de sesión limitado (~12 horas)
- Puede desconectarse por inactividad

---

**🌟 ¡Disfruta mejorando tus imágenes con IA!**

Si encuentras útil este notebook, considera darle una estrella al [repositorio original](https://github.com/philz1337x/clarity-upscaler) ⭐